In [1]:
#based on https://www.kaggle.com/concealberti/covid-week1-weather

In [11]:
import numpy as np
import pandas as pd

In [2]:
import os
os.environ["GOOGLE_APPLICATION_CREDENTIALS"]="/Users/lucas_botella/Downloads/tfm-covid-forecast-lb-278ff92b9d29.json"

In [3]:
# Set your own project id here
PROJECT_ID = 'poetic-bison-304616'
from google.cloud import bigquery
client = bigquery.Client(project=PROJECT_ID)

In [6]:
table1_stations = bigquery.TableReference.from_string(
    "bigquery-public-data.noaa_gsod.stations"
)
dataframe_stations = client.list_rows(
    table1_stations,
    selected_fields=[
        bigquery.SchemaField("usaf", "STRING"), #station number, world metherorological org
        bigquery.SchemaField("wban", "STRING"), #wban number, weather bureau army
        bigquery.SchemaField("country", "STRING"),
        bigquery.SchemaField("lat", "FLOAT"),
        bigquery.SchemaField("lon", "FLOAT"),
    ],
).to_dataframe()

dataframe_stations

/opt/anaconda3/lib/python3.8/site-packages/google/cloud/bigquery/client.py:444: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


,usaf,wban,country,lat,lon
0,007018,99999,None,0.000,0.000
1,007026,99999,AF,0.000,0.000
2,007070,99999,AF,0.000,0.000
3,008268,99999,AF,32.950,65.567
4,008307,99999,AF,0.000,0.000
...,...,...,...,...,...
29556,A00029,63820,US,36.380,-88.985
29557,A06854,00115,US,34.264,-116.854
29558,A51256,00451,US,36.699,-93.402
29559,720377,13917,US,35.633,-91.167


In [4]:
table1_gsod2020 = bigquery.TableReference.from_string(
    "bigquery-public-data.noaa_gsod.gsod2020"
)

dataframe_gsod2020= client.list_rows(table1_gsod2020,
    selected_fields=[
        bigquery.SchemaField("stn", "STRING"), #station number
        bigquery.SchemaField("wban", "STRING"), #station number
        bigquery.SchemaField("year", "INTEGER"),
        bigquery.SchemaField("mo", "INTEGER"),
        bigquery.SchemaField("da", "INTEGER"),
        bigquery.SchemaField("temp", "FLOAT"), #mean temp of the day
        bigquery.SchemaField("dewp", "FLOAT"), #mean_dew_point
        bigquery.SchemaField("slp", "FLOAT"), #mean_sealevel_pressure
        bigquery.SchemaField("wdsp", "FLOAT"), #mean_wind_speed
        bigquery.SchemaField("prcp", "FLOAT"), #total_precipitation
        bigquery.SchemaField("sndp", "FLOAT"), #snow_depth
    ],).to_dataframe()

dataframe_gsod2020

/opt/anaconda3/lib/python3.8/site-packages/google/cloud/bigquery/client.py:444: UserWarning: Cannot create BigQuery Storage client, the dependency google-cloud-bigquery-storage is not installed.
  warnings.warn(


,stn,wban,year,mo,da,temp,dewp,slp,wdsp,prcp,sndp
0,071200,99999,2020,12,11,46.7,9999.9,991.5,10.1,0.45,999.9
1,702220,26501,2020,1,7,-44.6,9999.9,1026.3,0.1,0.00,999.9
2,714850,99999,2020,10,16,49.7,9999.9,1020.6,24.5,1.69,999.9
3,712110,99999,2020,4,25,54.5,9999.9,1012.3,11.5,0.20,999.9
4,725287,04724,2020,9,3,69.6,9999.9,1007.3,8.4,0.00,999.9
...,...,...,...,...,...,...,...,...,...,...,...
4107416,994220,99999,2020,5,14,76.5,61.9,1019.3,11.1,0.00,999.9
4107417,994220,99999,2020,10,2,76.9,66.6,1014.2,7.6,0.00,999.9
4107418,994220,99999,2020,11,11,76.7,76.7,1006.9,18.7,0.00,999.9
4107419,997993,99999,2020,2,8,24.4,15.7,1008.7,14.5,0.54,999.9


In [165]:
train = pd.read_csv( '/Users/lucas_botella/Downloads/results/training_data_with_weather_info_week_1.csv')
train=train.rename(columns={"Lat": "lat", "Long": "lon"})
train

,Id,Province/State,Country/Region,lat,lon,Date,ConfirmedCases,Fatalities,day_from_jan_first,temp,min,max,stp,slp,dewp,rh,ah,wdsp,prcp,fog
0,1,NaN,Afghanistan,33.0000,65.0000,2020-01-22,0.0,0.0,22,42.6,33.6,54.9,999.9,1024.3,27.4,0.545709,0.186448,9.4,0.00,0
1,2,NaN,Afghanistan,33.0000,65.0000,2020-01-23,0.0,0.0,23,42.0,32.7,55.9,999.9,1020.8,22.8,0.461259,0.163225,14.9,99.99,1
2,3,NaN,Afghanistan,33.0000,65.0000,2020-01-24,0.0,0.0,24,40.1,36.9,43.2,999.9,1018.6,34.5,0.801794,0.325375,10.4,0.17,1
3,4,NaN,Afghanistan,33.0000,65.0000,2020-01-25,0.0,0.0,25,46.0,37.9,56.3,999.9,1018.0,37.8,0.728175,0.214562,6.1,0.57,1
4,5,NaN,Afghanistan,33.0000,65.0000,2020-01-26,0.0,0.0,26,42.8,36.1,53.1,999.9,1014.8,33.2,0.685513,0.231656,10.8,0.00,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
17887,26378,NaN,Zambia,-15.4167,28.2833,2020-03-20,2.0,0.0,80,70.6,62.6,81.9,999.9,NaN,62.8,0.761545,0.198068,6.0,0.00,0
17888,26379,NaN,Zambia,-15.4167,28.2833,2020-03-21,2.0,0.0,81,71.3,66.2,81.5,999.9,NaN,65.3,0.812047,0.212487,7.1,99.99,1
17889,26380,NaN,Zambia,-15.4167,28.2833,2020-03-22,3.0,0.0,82,72.1,67.1,80.4,999.9,NaN,66.7,0.829815,0.218712,5.0,99.99,1
17890,26381,NaN,Zambia,-15.4167,28.2833,2020-03-23,3.0,0.0,83,71.7,66.2,80.6,999.9,NaN,62.8,0.733343,0.192580,4.2,0.00,0


In [168]:
train.columns

Index(['Id', 'Province/State', 'Country/Region', 'lat', 'lon', 'Date',
       'ConfirmedCases', 'Fatalities', 'day_from_jan_first', 'temp', 'min',
       'max', 'stp', 'slp', 'dewp', 'rh', 'ah', 'wdsp', 'prcp', 'fog'],
      dtype='object')

In [7]:
stations_df= dataframe_stations
twenty_twenty_df= dataframe_gsod2020

In [171]:
stations_df['STN'] = stations_df['usaf'] + '-' + stations_df['wban']
twenty_twenty_df['STN'] = twenty_twenty_df['stn'] + '-' + twenty_twenty_df['wban']
cols_1= list(twenty_twenty_df.columns)
cols_2= list(stations_df.columns)
weather_df = twenty_twenty_df[cols_1].join(stations_df[cols_2].set_index('STN'), on='STN',  how='left', lsuffix='_left', rsuffix='_right')

weather_df['temp'] = weather_df['temp'].apply(lambda x: np.nan if x==9999.9 else x)
weather_df['slp'] = weather_df['slp'].apply(lambda x: np.nan if x==9999.9 else x)
weather_df['dewp'] = weather_df['dewp'].apply(lambda x: np.nan if x==9999.9 else x)
weather_df['wdsp'] = weather_df['wdsp'].apply(lambda x: np.nan if x==999.9 else x)
weather_df['prcp'] = weather_df['prcp'].apply(lambda x: np.nan if x==999.9 else x)
weather_df['sndp'] = weather_df['sndp'].apply(lambda x: np.nan if x==999.9 else x)

# convert everything into celsius
weather_df['temp'] = (weather_df['temp'] - 32) / 1.8
weather_df['dewp'] = (weather_df['dewp'] - 32) / 1.8
    
# compute relative humidity as ratio between actual vapour pressure (computed from dewpoint temperature)
# and saturation vapour pressure (computed from temperature) (the constant 6.1121 cancels out)
weather_df['rh'] = (np.exp((18.678*dewp)/(257.14+dewp))/np.exp((18.678*temp)/(257.14+temp)))

# calculate actual vapour pressure (in pascals)
# then use it to compute absolute humidity from the gas law of vapour 
# (ah = mass / volume = pressure / (constant * temperature))
weather_df['ah'] = ((np.exp((18.678*dewp)/(257.14+dewp))) * 6.1121 * 100) / (461.5 * temp)


weather_df['month']= weather_df['mo']
weather_df['day']= weather_df['da']
weather_df['Date']=pd.to_datetime(weather_df[['year','month','day']])
weather_df['Date2']= weather_df['Date']
weather_df['Date2']= weather_df['Date2'].astype('str')
mo2 = weather_df['Date2'].apply(lambda x: x[5:7])
da2 = weather_df['Date2'].apply(lambda x: x[8:10])
weather_df['day_from_jan_first'] = (da2.apply(int)
                               + 31*(mo2=='02') 
                               + 60*(mo2=='03')
                               + 91*(mo2=='04')  
                              )

In [172]:
weather_df

,stn,wban_left,year,mo,da,temp,dewp,slp,wdsp,prcp,...,country,lat,lon,rh,ah,month,day,Date,Date2,day_from_jan_first
0,071200,99999,2020,12,11,8.166667,NaN,991.5,10.1,0.45,...,FR,48.538,-2.854,NaN,NaN,12,11,2020-12-11,2020-12-11,11
1,702220,26501,2020,1,7,-42.555556,NaN,1026.3,0.1,0.00,...,US,64.737,-156.934,NaN,NaN,1,7,2020-01-07,2020-01-07,7
2,714850,99999,2020,10,16,9.833333,NaN,1020.6,24.5,1.69,...,CA,50.933,-127.650,NaN,NaN,10,16,2020-10-16,2020-10-16,16
3,712110,99999,2020,4,25,12.500000,NaN,1012.3,11.5,0.20,...,CA,49.483,-123.300,NaN,NaN,4,25,2020-04-25,2020-04-25,116
4,725287,04724,2020,9,3,20.888889,NaN,1007.3,8.4,0.00,...,US,43.108,-78.938,NaN,NaN,9,3,2020-09-03,2020-09-03,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4107416,994220,99999,2020,5,14,24.722222,16.611111,1019.3,11.1,0.00,...,US,27.070,-82.450,0.603578,0.166398,5,14,2020-05-14,2020-05-14,14
4107417,994220,99999,2020,10,2,24.944444,19.222222,1014.2,7.6,0.00,...,US,27.070,-82.450,0.702901,0.194650,10,2,2020-10-02,2020-10-02,2
4107418,994220,99999,2020,11,11,24.833333,24.833333,1006.9,18.7,0.00,...,US,27.070,-82.450,1.000000,0.276302,11,11,2020-11-11,2020-11-11,11
4107419,997993,99999,2020,2,8,-4.222222,-9.055556,1008.7,14.5,0.54,...,US,43.050,-70.833,0.690754,-0.158629,2,8,2020-02-08,2020-02-08,39


In [177]:
weather_df= weather_df.dropna(subset = ['lat', 'lon'])
weather_df = weather_df.reset_index(drop=True)
train= train.dropna(subset = ['lat', 'lon'])
train = train.reset_index(drop=True)
weather_df.lon= weather_df.lon.astype(int)
weather_df.lat= weather_df.lat.astype(int)
train.lon= train.lon.astype(int)
train.lat= train.lat.astype(int)

CovidWeather=train.merge(weather_df, on=['lat', 'lon', 'day_from_jan_first'], how='outer', indicator=True)
CovidWeather['temp_x'] = (CovidWeather['temp_x'] - 32) / 1.8
CovidWeather['dewp_x'] = (CovidWeather['temp_x'] - 32) / 1.8
CovidWeather

,Id,Province/State,Country/Region,lat,lon,Date_x,ConfirmedCases,Fatalities,day_from_jan_first,temp_x,...,usaf,wban_right,country,rh_y,ah_y,month,day,Date_y,Date2,_merge
0,1.0,NaN,Afghanistan,33,65,2020-01-22,0.0,0.0,22,5.888889,...,409420,99999,AF,0.210889,0.081562,10.0,22.0,2020-10-22,2020-10-22,both
1,2.0,NaN,Afghanistan,33,65,2020-01-23,0.0,0.0,23,5.555556,...,409420,99999,AF,0.440518,0.301783,10.0,23.0,2020-10-23,2020-10-23,both
2,2.0,NaN,Afghanistan,33,65,2020-01-23,0.0,0.0,23,5.555556,...,409420,99999,AF,0.757148,-0.036035,12.0,23.0,2020-12-23,2020-12-23,both
3,3.0,NaN,Afghanistan,33,65,2020-01-24,0.0,0.0,24,4.500000,...,409420,99999,AF,0.691457,-0.080483,12.0,24.0,2020-12-24,2020-12-24,both
4,3.0,NaN,Afghanistan,33,65,2020-01-24,0.0,0.0,24,4.500000,...,409420,99999,AF,0.367221,0.401653,10.0,24.0,2020-10-24,2020-10-24,both
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4107911,NaN,NaN,NaN,40,-93,NaN,NaN,NaN,113,NaN,...,725499,94991,US,0.495293,0.122385,4.0,22.0,2020-04-22,2020-04-22,right_only
4107912,NaN,NaN,NaN,28,-99,NaN,NaN,NaN,99,NaN,...,722526,12947,US,0.724371,0.211469,4.0,8.0,2020-04-08,2020-04-08,right_only
4107913,NaN,NaN,NaN,59,-149,NaN,NaN,NaN,47,NaN,...,994084,99999,US,0.629150,-0.623566,2.0,16.0,2020-02-16,2020-02-16,right_only
4107914,NaN,NaN,NaN,59,-149,NaN,NaN,NaN,49,NaN,...,994084,99999,US,0.988209,0.584682,2.0,18.0,2020-02-18,2020-02-18,right_only


In [219]:
train_countries=CovidWeather[['Country/Region', 'country']]
train_countries.drop_duplicates()
train_countries[train_countries['Country/Region']=='Spain']

,Country/Region,country
65390,Spain,SP
65391,Spain,SP
65392,Spain,SP
65393,Spain,SP
65394,Spain,SP
...,...,...
65800,Spain,SP
65801,Spain,SP
65802,Spain,SP
65803,Spain,SP


In [129]:
CovidWeather['Country/Region'].unique()

array(['Afghanistan', 'Albania', 'Algeria', 'Andorra',
       'Antigua and Barbuda', 'Argentina', 'Armenia', 'Aruba',
       'Australia', 'Austria', 'Azerbaijan', 'Bahrain', 'Bangladesh',
       'Barbados', 'Belarus', 'Belgium', 'Benin', 'Bhutan', 'Bolivia',
       'Bosnia and Herzegovina', 'Brazil', 'Brunei', 'Bulgaria',
       'Burkina Faso', 'Cambodia', 'Cameroon', 'Canada',
       'Central African Republic', 'Chile', 'China', 'Colombia',
       'Congo (Brazzaville)', 'Congo (Kinshasa)', 'Costa Rica',
       "Cote d'Ivoire", 'Croatia', 'Cruise Ship', 'Cuba', 'Cyprus',
       'Czechia', 'Denmark', 'Djibouti', 'Dominican Republic', 'Ecuador',
       'Egypt', 'Equatorial Guinea', 'Estonia', 'Eswatini', 'Ethiopia',
       'Finland', 'France', 'French Guiana', 'Gabon', 'Gambia, The',
       'Georgia', 'Germany', 'Ghana', 'Greece', 'Greenland', 'Guadeloupe',
       'Guam', 'Guatemala', 'Guernsey', 'Guinea', 'Guyana', 'Holy See',
       'Honduras', 'Hungary', 'Iceland', 'India', 'Indonesia

In [201]:
df_covid_weather_grouped = CovidWeather.groupby(['country', pd.Grouper(key='Date_y', freq='W-MON')]).mean().reset_index()

In [202]:
df_covid_weather_grouped

,country,Date_y,Id,lat,lon,ConfirmedCases,Fatalities,day_from_jan_first,temp_x,min,...,temp_y,dewp_y,slp_y,wdsp_y,prcp_y,sndp,rh_y,ah_y,month,day
0,AA,2020-01-06,NaN,12.00,-70.00,NaN,NaN,3.5,NaN,NaN,...,27.361111,22.129630,1014.366667,11.033333,0.005000,NaN,0.730058,0.212896,1.000000,3.500000
1,AA,2020-01-13,NaN,12.00,-70.00,NaN,NaN,10.0,NaN,NaN,...,27.309524,22.174603,1014.800000,16.428571,0.121429,NaN,0.734130,0.213876,1.000000,10.000000
2,AA,2020-01-20,NaN,12.00,-70.00,NaN,NaN,17.0,NaN,NaN,...,27.222222,22.420635,1013.542857,18.228571,0.052857,NaN,0.748419,0.217668,1.000000,17.000000
3,AA,2020-01-27,15348.5,12.00,-70.00,0.0,0.0,24.0,26.694444,74.750000,...,26.761905,22.634921,1012.457143,9.157143,0.020000,NaN,0.779462,0.224365,1.000000,24.000000
4,AA,2020-02-03,15355.0,12.00,-70.00,0.0,0.0,31.0,27.595238,77.342857,...,27.595238,22.857143,1013.557143,13.942857,0.000000,NaN,0.751829,0.220533,1.428571,17.714286
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12311,ZI,2020-09-14,NaN,-17.00,31.00,NaN,NaN,10.8,NaN,NaN,...,19.655556,2.988889,NaN,7.380000,0.000000,NaN,0.354061,0.090542,9.000000,10.800000
12312,ZI,2020-09-21,NaN,-17.00,31.00,NaN,NaN,18.0,NaN,NaN,...,23.379630,5.342593,NaN,8.233333,0.000000,NaN,0.333017,0.088962,9.000000,18.000000
12313,ZI,2020-09-28,NaN,-17.00,31.00,NaN,NaN,22.0,NaN,NaN,...,17.666667,8.277778,NaN,10.200000,0.000000,NaN,0.538892,0.134231,9.000000,22.000000
12314,ZI,2020-11-02,NaN,-17.75,30.25,NaN,NaN,29.0,NaN,NaN,...,26.694444,7.250000,NaN,8.775000,0.000000,NaN,0.297114,0.084925,10.000000,29.000000


In [233]:
df_covid_weather_grouped[(df_covid_weather_grouped['country']=='SP') & (df_covid_weather_grouped['month']==12)]

,country,Date_y,Id,lat,lon,ConfirmedCases,Fatalities,day_from_jan_first,temp_x,min,...,temp_y,dewp_y,slp_y,wdsp_y,prcp_y,sndp,rh_y,ah_y,month,day
10143,SP,2020-12-07,NaN,38.534759,-3.878342,NaN,NaN,3.979947,NaN,NaN,...,8.647504,3.971628,1011.935443,7.418937,4.652420,1.607143,0.736144,inf,12.0,3.979947
10144,SP,2020-12-14,NaN,38.541333,-3.881333,NaN,NaN,11.000000,NaN,NaN,...,11.346889,7.351630,1014.598729,7.711821,5.480733,1.833333,0.777123,0.227345,12.0,11.000000
10145,SP,2020-12-21,NaN,38.525606,-3.907008,NaN,NaN,17.994609,NaN,NaN,...,10.420635,7.428604,1021.153219,4.771703,3.993127,0.960000,0.825399,0.258991,12.0,17.994609
10146,SP,2020-12-28,16297.6,38.560748,-3.862483,0.0,0.0,24.989319,10.479365,46.588571,...,8.711467,3.923174,1020.637607,6.640000,3.128545,0.685714,0.734982,0.279070,12.0,24.989319
10147,SP,2021-01-04,16302.6,38.541796,-3.882353,0.0,0.0,29.996904,13.325926,50.046667,...,6.719814,1.878053,1014.261765,6.940063,5.677059,1.625000,0.728307,0.193549,12.0,29.996904


In [155]:
CovidWeather[CovidWeather['Date_y'].dt.month==2]['Date_y'].dt.month.unique()

array([2])